<img src="img/se4g_pic1.png">

In [ ]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
import pandas as pd
import os
from datetime import datetime
from se4g_helper import *
import ipywidgets as widgets


Load dashboard table

In [ ]:
def selection_from_source_table(table_name,conn):
    
    
    cursor = conn.cursor()

    # Generate the SQL statement to select data from the source table
    select_data_query = f"SELECT * FROM {table_name};"

    # Execute the SELECT command
    cursor.execute(select_data_query)

    columns = [desc[0] for desc in cursor.description]

    # Fetch all the rows
    rows = cursor.fetchall()

    cursor.close()
    conn.close()

    # Create a pandas DataFrame from the fetched rows
    df = pd.DataFrame(rows, columns=columns)

    return df

In [ ]:
import psycopg2

def connect_to_database(host, port, database, user, password):
    """
    Establishes a connection to a PostgreSQL database.

    Args:
        host (str): Hostname or IP address of the database server.
        port (str): Port number for the database server.
        database (str): Name of the database.
        user (str): Username for database authentication.
        password (str): Password for database authentication.

    Returns:
        connection (psycopg2.extensions.connection): PostgreSQL database connection object.
    """
    try:
        connection = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
        print("Database connection established successfully.")
        return connection
    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")

# Example usage
conn = connect_to_database('your_host', 'your_port', 'your_database', 'your_user', 'your_password')

#Funzionerà?  -----> bello con il try !!!






In [ ]:
#from this I will obtain a DF that contains all the columns and the rows of our table pollutant_detection
conn = connect_right_now()

table_name='pollutant_detection'

df_pollutant=selection_from_source_table(table_name,conn)

In [ ]:
df_pollutant.head()

In [ ]:
from datetime import datetime
import pandas as pd

df_pollutant['date_time_end'] = pd.to_datetime(df_pollutant['date_time_end'], format='%Y-%m-%d %H:%M:%S%z')

day = df_pollutant['date_time_end'].dt.day

In [ ]:
datetime_objects = df_pollutant['date_time_end'][0:10].apply(lambda x: x.to_pydatetime())

formatted_datetimes = datetime_objects.apply(lambda x: x.strftime('%m-%d_%H'))

print(f"Formatted datetime objects: {formatted_datetimes.max()}")

In [ ]:
#from this I will obtain a DF that contains all the columns and the rows of our table station
table_name = 'station'

df_station=selection_from_source_table(table_name,conn)

In [ ]:

df_station.head()

In [ ]:
# Dropdown widgets for country and pollutant selection
country_dropdown = widgets.Dropdown(
    options=df_station['network_country'].unique(),
    description='Country:'
)

pollutant_dropdown = widgets.Dropdown(
    description='Pollutant:'
)

# Text widget to display statistics
statistics_text = widgets.Textarea(
    description='Statistics:',
    disabled=True,
    layout={'width': '500px', 'height': '200px'}
)

# Function to update pollutants based on selected country
def update_pollutants(change):
    country = country_dropdown.value
    
    if country:
        pollutants = df_pollutant[df_pollutant['station_code'].isin(df_station[df_station['network_country'] == country]['station_code'])]['pollutant'].unique()
        pollutant_dropdown.options = pollutants
        pollutant_dropdown.disabled = False
    else:
        pollutant_dropdown.options = []
        pollutant_dropdown.disabled = True

# Function to update statistics based on user selection
def update_statistics(change):
    country = country_dropdown.value
    pollutant = pollutant_dropdown.value
    
    filtered_df = df_pollutant.merge(df_station, on='station_code')
    filtered_df = filtered_df[(filtered_df['network_country'] == country) & (filtered_df['pollutant'] == pollutant)]
    
    mean_value = filtered_df['value_numeric'].mean()
    max_value = filtered_df['value_numeric'].max()
    min_value = filtered_df['value_numeric'].min()
    
    statistics_text.value = f"Mean: {mean_value:.2f}\nMax: {max_value}\nMin: {min_value}"

# Event listeners for dropdown selection
country_dropdown.observe(update_pollutants, 'value')
pollutant_dropdown.observe(update_statistics, 'value')

In [ ]:
import matplotlib.pyplot as plt

def plot_statistics(change):
    country = country_dropdown.value
    pollutant = pollutant_dropdown.value
    
    filtered_df = df_pollutant.merge(df_station, on='station_code')
    filtered_df = filtered_df[(filtered_df['network_country'] == country) & (filtered_df['pollutant'] == pollutant)]
    
    mean_value = filtered_df['value_numeric'].mean()
    max_value = filtered_df['value_numeric'].max()
    min_value = filtered_df['value_numeric'].min()
    
    statistics_text.value = f"Mean: {mean_value:.2f}\nMax: {max_value}\nMin: {min_value}"
    
    # Create a bar plot of the statistics
    stats = [mean_value, max_value, min_value]
    labels = ['Mean', 'Max', 'Min']
    
    plt.figure(figsize=(8, 6))
    plt.bar(labels, stats)
    plt.xlabel('Statistic')
    plt.ylabel('Value')
    plt.title(f'Statistics for {pollutant} in {country}')
    plt.show()

# Event listener for dropdown selection
pollutant_dropdown.observe(plot_statistics, 'value')

# Display widgets
display(country_dropdown)
display(pollutant_dropdown)
display(statistics_text)

Construct the app and callbacks

In [3]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
 
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='SO2'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='CO'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Andorra'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['day'].min(),
        max=df['day'].max(),
        value=df['day'].max(),
        marks={str(hour): str(hour) for hour in df['day'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['day'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['pollutant'] == xaxis_column_name]['value_numeric_mean'],
            y=dff[dff['pollutant'] == yaxis_column_name]['value_numeric_mean'],
            text=dff[dff['pollutant'] == yaxis_column_name]['country'],
            customdata=dff[dff['pollutant'] == yaxis_column_name]['country'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }


def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['day'],
            y=dff['value_numeric_mean'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }


@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['country'] == country_name]
    dff = dff[dff['pollutant'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

 
@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['country'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['pollutant'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

Serve the app using `run_server`.  Unlike the standard `Dash.run_server` method, the `JupyterDash.run_server` method doesn't block execution of the notebook. It serves the app in a background thread, making it possible to run other notebook calculations while the app is running.

This makes it possible to iteratively update the app without rerunning the potentially expensive data processing steps.

In [4]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


By default, `run_server` displays a URL that you can click on to open the app in a browser tab. The `mode` argument to `run_server` can be used to change this behavior.  Setting `mode="inline"` will display the app directly in the notebook output cell.

In [5]:
app.run_server(mode="inline")

Dash is running on http://127.0.0.1:8050/



When running in JupyterLab, with the `jupyterlab-dash` extension, setting `mode="jupyterlab"` will open the app in a tab in JupyterLab.

In [7]:
app.run_server(mode="jupyterlab")

Dash is running on http://127.0.0.1:8050/



To run a notebook from another notebook you can use the magic function:
    
```
%run .../other_notebook.ipynb
```

<img src="img/se4g_pic2.png">